## Part 2 : REINFORCE algorithm

In [6]:
import gymnasium as gym
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import pygame
import scipy as sc
from collections import deque
from tqdm import *

## Description

The system consists of two links connected linearly to form a chain, with one end of the chain fixed. The joint between the two links is actuated. The goal is to apply torques on the actuated joint to swing the free end of the linear chain above a given height while starting from the initial state of hanging downwards.

## Action Space
The action is discrete, deterministic, and represents the torque applied on the actuated joint between the two links.
    | Num | Action                                | Unit         |
    |-----|---------------------------------------|--------------|
    | 0   | apply -1 torque to the actuated joint | torque (N m) |
    | 1   | apply 0 torque to the actuated joint  | torque (N m) |
    | 2   | apply 1 torque to the actuated joint  | torque (N m) 

## Observation Space
The observation is a ndarray with shape (6,) that provides information about the two rotational joint angles as well as their angular velocities.

    | Num | Observation                  | Min                 | Max               |
    |-----|------------------------------|---------------------|-------------------|
    | 0   | Cosine of `theta1`           | -1                  | 1                 |
    | 1   | Sine of `theta1`             | -1                  | 1                 |
    | 2   | Cosine of `theta2`           | -1                  | 1                 |
    | 3   | Sine of `theta2`             | -1                  | 1                 |
    | 4   | Angular velocity of `theta1` | ~ -12.567 (-4 * pi) | ~ 12.567 (4 * pi) |
    | 5   | Angular velocity of `theta2` | ~ -28.274 (-9 * pi) | ~ 28.274 (9 * pi) |

where

theta1 is the angle of the first joint, where an angle of 0 indicates the first link is pointing directly downwards.

theta2 is relative to the angle of the first link. An angle of 0 corresponds to having the same angle between the two links.

The angular velocities of theta1 and theta2 are bounded at ±4π, and ±9π rad/s respectively. A state of [1, 0, 1, 0, ..., ...] indicates that both links are pointing downwards.

## Rewards
The goal is to have the free end reach a designated target height in as few steps as possible, and as such all steps that do not reach the goal incur a reward of -1. Achieving the target height results in termination with a reward of 0. The reward threshold is -100.

## Starting State
Each parameter in the underlying state (theta1, theta2, and the two angular velocities) is initialized uniformly between -0.1 and 0.1. This means both links are pointing downwards with some initial stochasticity.

## Episode End
The episode ends if one of the following occurs:

1. Termination: The free end reaches the target height, which is constructed as: -cos(theta1) - cos(theta2 + theta1) > 1.0

2. Truncation: Episode length is greater than 500 (200 for v0)

In [ ]:
env = gym.make('Acrobot-v1')

In [ ]:
# To change the dynamics as described above
env.env.book_or_nips = 'nips'

In [ ]:
n_actions = env.action_space.n
shape_states = env.observation_space.shape
print(n_actions)
print(shape_states)

In [10]:
def policy(state,theta): #softmax
    p=state @ theta
    return sc.special.softmax(p)

def gradient_function(state,theta):
    z=[0,0,0]
    for i in range(3):
        z[i]=np.dot(state,1-policy(state,theta)[i])
    return np.array(z)

In [66]:
def reinforce(theta_0,lr,gamma,n_episode):
    env = gym.make("Acrobot-v1") #, render_mode="human"
    theta=theta_0
    History=[]
    
    for i in range(n_episode):
        X=[] #list of states
        A=[] #list of actions
        R=[] #list of rewards
        Grad=[] #list of gradients
        x,_=env.reset()
        n_move = 0 
        terminated=False
        truncated=False
        while not terminated and not truncated: #episode to fill the lists
            #if n_move > 500:
             #   env.close()
              #  raise Exception("Too many attempts, failed")
            n_move += 1
            X.append(x)
            pol=policy(x, theta)
            #print(pol)
            action=np.random.choice([0,1,2], p=pol)
            #print(action)
            A.append(action)
            Grad.append(gradient_function(x,theta)[action, :])
            x, r, terminated, truncated, info = env.step(action)
            R.append(r)
        #print(n_move)
        History.append(np.sum(R))
        
        for n in range(n_move): #list run for the adjustment of theta
            Rewardstot=np.cumsum(R[n:n_move])
            G=Rewardstot[-1]
            theta[:,A[n]]=theta[:,A[n]]+lr*G*Grad[n]
    #env.close()
    return theta, History

In [68]:
lr=0.001
n_episode=100
gamma=1
U=np.random.uniform(-0.1,0.1,18)
theta_0=U.reshape((6,3))

In [67]:
theta_0

array([[-125.52624156, -124.33188712, -125.90155508],
       [  -0.96981579,   -1.85637498,   -2.63198043],
       [-103.29190925, -102.63544755, -103.32686696],
       [   5.61705568,    3.84433235,    4.64714133],
       [  16.02271467,   17.78390388,   14.45906153],
       [ -25.7340427 ,  -32.28577782,  -22.76946646]])

In [69]:
theta_opt, History=reinforce(theta_0,lr,gamma,n_episode)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(History)